In [1]:
leafEnd = -1
class Node:
    def __init__(self, leaf):
        # self.__identifier = identifier
        self.children = {}
        # for leaf nodes, it stores the index of suffix for
        # the path  from root to leaf"""
        self.leaf = leaf
        #stores the index of the suffix link
        self.suffixIndex = None    
        #start and end specifies the edge by which the 2 nodes are connected and this indices are stored in end node
        self.start = None
        self.end = None
        self.suffixLink = None

    def __getattribute__(self, name):
        if name == 'end':
            if self.leaf:
                return leafEnd
        return super(Node, self).__getattribute__(name)


class SuffixTree:
    def __init__(self, data):
        self._string = data + '$'
        #lastNewNode will point to newly created internal node,waiting for it's suffix link to be set, which might get
        #a new suffix link (other than root) in next extension of same phase. lastNewNode will be set to NULL when last
        #newly created internal node got suffix link reset to new internal node created in next
        #extension of same phase.
        self.lastNewNode = None
        #activenode could be root node or internal node
        self.activeNode = None
        #activeEdge points to position of input string character index
        self.activeEdge = -1
        self.activeLength = 0
        self.remainingSuffixCount = 0
        self.rootEnd = None
        self.splitEnd = None
        self.size = -1  # Length of input string
        self.root = None #root node


    def edge_length(self, node):
        return node.end - node.start + 1

    def walk_down(self, current_node):
        length = self.edge_length(current_node)
#active length tells how many charatcers we need to traverse from active node 
#skip/count trick
#If activeLength is greater than current edge length, set next internal node as activeNode and adjust activeEdge and activeLength
        if (self.activeLength >= length):
            #When we are on root node or internal node and we need to walk down, we need to know which edge to choose. 
            self.activeEdge += length
            self.activeLength -= length
            self.activeNode = current_node
            return True
        return False
    
    def new_node(self, start, end=None, leaf=False):
        node = Node(leaf)
        #for root node suffix link will be set to zero
        #for internal node suffix link will be set to root and it changes as internal node increases
        node.suffixLink = self.root
        node.start = start
        node.end = end
        #by defualt it is -1 by default and actual suffix index will be set later for leaves at the end of all phases
        node.suffixIndex = -1
        return node

    def extend_suffix_tree(self, pos):
        #all leaf edge end indices will point to this global variable
        global leafEnd
        #After completing phase till leaf edges this will be incremented
        leafEnd = pos
        #we are incrementing since suffix needs to be added
        self.remainingSuffixCount += 1
        #we will set it to none when starting new phase showing that there are no internal nodes
        self.lastNewNode = None
        # Add all suffixes (yet to be added) one by one in tree
        while(self.remainingSuffixCount > 0):
            if (self.activeLength == 0):
                self.activeEdge = pos  # APCFALZ
#There is no outgoing edge starting with activeEdge from activeNode so we create a new leaf edge
            if (self.activeNode.children.get(self._string[self.activeEdge]) is None):
                self.activeNode.children[self._string[self.activeEdge]] = self.new_node(pos, leaf=True)
#if its not last node then reset suffixlink from last node to current active node and set last new node to zero
                if (self.lastNewNode is not None):
                    self.lastNewNode.suffixLink = self.activeNode
                    self.lastNewNode = None
            #  There is an outgoing edge starting with activeEdge from activeNode
            else:
                #  Get the next node at the end of edge starting with activeEdge
                _next = self.activeNode.children.get(self._string[self.activeEdge])
                if self.walk_down(_next):  # Do walkdown
                    # Start from _next node (the new activeNode)
                    continue
                    #extension rule 3
                    #current character being processed is already on the edge
                if (self._string[_next.start + self.activeLength] == self._string[pos]):
# If a newly created node waiting for it's suffix link to be set, then set suffix link of that waiting node to curent. active node
                    if((self.lastNewNode is not None) and (self.activeNode != self.root)):
                        self.lastNewNode.suffixLink = self.activeNode
                        self.lastNewNode = None
                    # APCFER3
                    self.activeLength += 1
                    break
                self.splitEnd = _next.start + self.activeLength - 1
                # New internal node
                split = self.new_node(_next.start, self.splitEnd)
                self.activeNode.children[self._string[self.activeEdge]] = split
                # New leaf coming out of new internal node
                split.children[self._string[pos]] = self.new_node(pos, leaf=True)
                _next.start += self.activeLength
                split.children[self._string[_next.start]] = _next
#We got a new internal node here. If there is any internal node created in last extensions of same
#phase which is still waiting for it's suffix link reset, do it now
                if (self.lastNewNode is not None):
                    self.lastNewNode.suffixLink = split
                self.lastNewNode = split
#after 1 suffix got added to the tree decrement the count of suffix which are yet to add
            self.remainingSuffixCount -= 1
            if ((self.activeNode == self.root) and (self.activeLength > 0)):  # APCFER2C1
                self.activeLength -= 1
                self.activeEdge = pos - self.remainingSuffixCount + 1
            elif (self.activeNode != self.root):  # APCFER2C2
                self.activeNode = self.activeNode.suffixLink

    def walks(self, current):
        start, end = current.start, current.end
        yield self._string[start: end + 1]

        for node in current.children.values():
            if node:
                yield from self.walks(node)
    

    def build_suffix_tree(self):
        self.size = len(self._string)
        self.rootEnd = -1
        self.root = self.new_node(-1, self.rootEnd)
        self.activeNode = self.root  # First activeNode will be root
        for i in range(self.size):
            self.extend_suffix_tree(i)

    def print_dfs(self):
        for sub in self.walks(self.root):
            print(sub)

In [2]:
class CheckSubString:
    def __init__(self, tree, sub_string, findall=False):
        self.tree = tree
        self.main_string = tree._string
        self.root = tree.root
        self.sub_string = sub_string
        self.latest_index = 0
        self.findall = findall
        self.continue_flag = False
        self.sub_length = len(sub_string)

    def traverse(self, node, sub_string):
        if sub_string:
            # Since each child starts with a unique character we will pursue the process for the child that sub-string
            # Starts with the frist character of this edge
            item = next(((char, child) for char, child in node.children.items() if sub_string.startswith(char)), None)

            if item:
                char, child = item
                start, end = child.start, child.end
                # If the edge is equal with sub-string returns the index
                if self.main_string[start: end + 1].startswith(sub_string):
                    if self.findall:
                        return self.find_all_match(child, len(sub_string))
                    return start - (self.sub_length - len(sub_string))
                # sub-string starts with the frist character of our edge but is not equal with it
                # So call the travese for the rest of sub-string (from the lenght of previous edge)
                return self.traverse(child, sub_string[end - start + 1:])
            else:
                # At this level there were no edge that sub-string starts with its leading character.
                return -1
        if self.findall:
            return self.find_all_match(node, len(sub_string))
        return node.start - (self.sub_length - len(sub_string))

    def check(self):
        if self.root is None:
            return -1
        if not isinstance(self.sub_string, str):
            return -1
        if not self.sub_string:
            # Every string starts with an empty string
            return 0

        return self.traverse(self.root, self.sub_string)

    def find_all_match(self, node, sub_length):

        def inner(node, traversed_edges):
            for char, child in node.children.items():
                if child.leaf:
                    yield child.start - traversed_edges
                else:
                    start, end = child.start, child.end
                    sub_length = end - start + 1
                    yield from inner(child, traversed_edges + sub_length)

        if node.leaf:
            first = node.start - (self.sub_length - sub_length)
            return [first, *inner(node, self.sub_length)]
        else:
            return list(inner(node, self.sub_length))

In [3]:
if __name__ == '__main__':
    with open('human.txt','r') as file:
        #string = [line.strip() for line in file]
        p = []
        for line in file:
            data = line.split()
            p.append(data)
        n = int(input("Enter the sequence"))
        for i in range(n):
            s = p[n][0]
        val = input("Enter String ")
        tree = SuffixTree(s)
        tree.build_suffix_tree()
        a = CheckSubString(tree, val, findall=True)
        print(a.check())
        tree.print_dfs()
    
    

Enter the sequence1
Enter String AAT
[28, 5]

A
C
C
ATAAT$
GTATGGCCCACCATAAT$
TA
CCGTATGGCCCACCATAAT$
AATACTACCGTATGGCCCACCATAAT$
A
ATACTACCGTATGGCCCACCATAAT$
CTAAATACTACCGTATGGCCCACCATAAT$
T
$
ACTACCGTATGGCCCACCATAAT$
T
GGCCCACCATAAT$
A
AT$
CTACCGTATGGCCCACCATAAT$
$
C
C
CACCATAAT$
GTATGGCCCACCATAAT$
A
TAAT$
CCATAAT$
TA
CCGTATGGCCCACCATAAT$
AATACTACCGTATGGCCCACCATAAT$
GTATGGCCCACCATAAT$
A
TAAT$
CCATAAT$
T
GGCCCACCATAAT$
A
C
CGTATGGCCCACCATAAT$
TACCGTATGGCCCACCATAAT$
A
T$
ATACTACCGTATGGCCCACCATAAT$
TGGCCCACCATAAT$
$
G
GCCCACCATAAT$
TATGGCCCACCATAAT$
CCCACCATAAT$
$
